In [1]:
! pip install pyspark

In [4]:
import pyspark

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark=SparkSession.builder.appName('session4').getOrCreate()

In [9]:
spark

In [2]:
df_pyspark=spark.read.csv('pyspark.csv',header=True,inferSchema=True)
df_pyspark.show()

NameError: name 'spark' is not defined

In [18]:
df_pyspark1=spark.read.option('header','true').csv('Book1.csv',inferSchema=True)
df_pyspark1.show()

+-------+----+----+
|   name| age| exp|
+-------+----+----+
|veeresh|  25|   4|
|sachin |  24|   3|
|   asas|  26|   2|
| ggajdj|  29|   6|
|hdhdjah|null|null|
|   ahaj|  30|null|
+-------+----+----+



In [20]:
df_pyspark.drop('exp').show()

+-------+----+
|   name| age|
+-------+----+
|veeresh|  25|
|sachin |  24|
|   asas|  26|
| ggajdj|  29|
|hdhdjah|null|
|   ahaj|  30|
+-------+----+



In [24]:
df_pyspark.drop('age','exp').show()

+-------+
|   name|
+-------+
|veeresh|
|sachin |
|   asas|
| ggajdj|
|hdhdjah|
|   ahaj|
+-------+



In [26]:
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- exp: integer (nullable = true)



In [27]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [31]:
df_pyspark.select('name').show()

+-------+
|   name|
+-------+
|veeresh|
|sachin |
|   asas|
| ggajdj|
|hdhdjah|
|   ahaj|
+-------+



In [33]:
df_pyspark.select('name','age','exp').show()

+-------+----+----+
|   name| age| exp|
+-------+----+----+
|veeresh|  25|   4|
|sachin |  24|   3|
|   asas|  26|   2|
| ggajdj|  29|   6|
|hdhdjah|null|null|
|   ahaj|  30|null|
+-------+----+----+



In [40]:
df_pyspark.describe

<bound method DataFrame.describe of DataFrame[name: string, age: int, exp: int]>

In [45]:
df_pyspark.na.drop(how='all',thresh=1).show()

+-------+----+----+
|   name| age| exp|
+-------+----+----+
|veeresh|  25|   4|
|sachin |  24|   3|
|   asas|  26|   2|
| ggajdj|  29|   6|
|hdhdjah|null|null|
|   ahaj|  30|null|
+-------+----+----+



In [50]:
df_pyspark.withColumn('exp2',df_pyspark['exp']+2).show()

+-------+----+----+----+
|   name| age| exp|exp2|
+-------+----+----+----+
|veeresh|  25|   4|   6|
|sachin |  24|   3|   5|
|   asas|  26|   2|   4|
| ggajdj|  29|   6|   8|
|hdhdjah|null|null|null|
|   ahaj|  30|null|null|
+-------+----+----+----+



In [54]:
df_pyspark.withColumnRenamed('exp','exp2').show()

+-------+----+----+
|   name| age|exp2|
+-------+----+----+
|veeresh|  25|   4|
|sachin |  24|   3|
|   asas|  26|   2|
| ggajdj|  29|   6|
|hdhdjah|null|null|
|   ahaj|  30|null|
+-------+----+----+



In [58]:
df_pyspark.na.fill('missing values','exp').show()

+-------+----+----+
|   name| age| exp|
+-------+----+----+
|veeresh|  25|   4|
|sachin |  24|   3|
|   asas|  26|   2|
| ggajdj|  29|   6|
|hdhdjah|null|null|
|   ahaj|  30|null|
+-------+----+----+



In [59]:
from pyspark.ml.feature import Imputer

In [64]:
imputer = Imputer(
    inputCols=['age','exp'], 
    outputCols=["{}_imputed".format(c) for c in [ 'age', 'exp']]
    ).setStrategy("mean")
              

In [70]:
df_pyspark=imputer.fit(df_pyspark).transform(df_pyspark)

In [72]:
df_pyspark=df_pyspark.drop('age','exp')

## filter operations

In [73]:
df_pyspark.show()

+-------+-----------+-----------+
|   name|age_imputed|exp_imputed|
+-------+-----------+-----------+
|veeresh|         25|          4|
|sachin |         24|          3|
|   asas|         26|          2|
| ggajdj|         29|          6|
|hdhdjah|         26|          3|
|   ahaj|         30|          3|
+-------+-----------+-----------+



In [90]:
df_pyspark.filter(("age_imputed<=26")).show()

+-------+-----------+-----------+
|   name|age_imputed|exp_imputed|
+-------+-----------+-----------+
|veeresh|         25|          4|
|sachin |         24|          3|
|   asas|         26|          2|
|hdhdjah|         26|          3|
+-------+-----------+-----------+



In [88]:
df_pyspark.filter(("age_imputed<=26")).select('name').show()

+-------+
|   name|
+-------+
|veeresh|
|sachin |
|   asas|
|hdhdjah|
+-------+



In [86]:
df_pyspark.filter((df_pysaprk["age_imputed"]<=26)
                  (df_pyspark["age_imputed"]>=20)).show()

AnalysisException: Resolved attribute(s) age_imputed#744 missing from name#38,age_imputed#802,exp_imputed#803 in operator !Filter ((age_imputed#744 <= 26) AND (age_imputed#802 >= 20)). Attribute(s) with the same name appear in the operation: age_imputed. Please check if the right attribute(s) are used.;
!Filter ((age_imputed#744 <= 26) AND (age_imputed#802 >= 20))
+- Project [name#38, age_imputed#802, exp_imputed#803]
   +- Project [name#38, age#39, exp#40, cast(CASE WHEN isnull(cast(age#39 as double)) THEN 26.8 WHEN (cast(age#39 as double) = NaN) THEN 26.8 ELSE cast(age#39 as double) END as int) AS age_imputed#802, cast(CASE WHEN isnull(cast(exp#40 as double)) THEN 3.75 WHEN (cast(exp#40 as double) = NaN) THEN 3.75 ELSE cast(exp#40 as double) END as int) AS exp_imputed#803]
      +- Relation[name#38,age#39,exp#40] csv
